<a href="https://colab.research.google.com/github/ram130849/Deep_Learning_Systems_Assignments/blob/main/TensorFlow/Ramki/E533_DLS_Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install IPython
!pip install librosa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 12.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [52]:
import numpy as np
from IPython.display import Audio
import librosa
from google.colab import drive
import tensorflow as tf
from tensorflow.keras import backend as K
from librosa.core import stft,istft
from tensorflow.keras.layers import GRU,Dropout
from tensorflow.keras.layers import Conv1D,Conv2D,MaxPooling2D,Dropout,Dense,Flatten,BatchNormalization
from sklearn.cross_validation import train_test_split
import matplotlib.pyplot as plt
import os
import soundfile as sf
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Q1: Data Augmentation

In [ ]:
cifar10 = tf.keras.datasets.cifar10

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
 X_train, X_val, Y_train, Y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=1)

In [ ]:
X_train, X_val = X_train/255.0, X_val/ 255.0
Y_train, Y_val = Y_train.flatten(), Y_val.flatten()``

In [ ]:
he_initializer = tf.keras.initializers.HeUniform(seed=None)

In [ ]:
model = models.Sequential()
model.add(Conv2D(10, (5, 5), strides=(1,1), activation='relu', input_shape=(32, 32, 3),kernel_initializer=he_initializer))
model.add(MaxPooling2D((2, 2),strides=(2,2))
model.add(Conv2D(10, (5, 5), activation='relu',kernel_initializer=he_initializer))
model.add(MaxPooling2D((2, 2),strides=2))
model.add(Flatten())
model.add(Dense(units=20,activation='relu',kernel_initializer=he_initializer))
model.add(Dense(units=10,activation='softmax',kernel_initializer=he_initializer))

In [ ]:
adam = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=adam,loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
r = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=200)

In [6]:
# !unzip '/content/drive/MyDrive/DLS_DATA/homework3.zip' -d '/content/drive/MyDrive/DLS_DATA/timit-hw3'

In [65]:
path = '/content/drive/MyDrive/DLS_DATA/timit-hw3/timit-homework/tr'

X = []
S = []
N = []

for f in os.listdir(path):
    if(os.path.isfile(os.path.join(path,f))):
        if(f[:3]=='trn'):
          s, sr = librosa.load(os.path.join(path,f),sr=None)
          trn_stft=librosa.stft(s, n_fft=1024, hop_length=512)
          N.append(abs(trn_stft))
        elif(f[:3]=='trs'):
          s, sr = librosa.load(os.path.join(path,f),sr=None)
          trs_stft=librosa.stft(s, n_fft=1024, hop_length=512)
          S.append(abs(trs_stft))
        else:
          s, sr = librosa.load(os.path.join(path,f),sr=None)
          trx_stft=librosa.stft(s, n_fft=1024, hop_length=512)
          X.append(abs(trx_stft))

In [66]:
print(N[-1].shape)
print(S[-1].shape)
print(X[-1].shape)

(513, 105)
(513, 95)
(513, 111)


In [67]:
max_len = X[0].shape[1]
for k in range(1,len(X)):
    if(X[k].shape[1]>max_len):
        max_len = X[k].shape[1]
print(max_len)

178


In [68]:
M_arr = []
for i in range(len(N)):
    # print("Before:",S[i].shape,N[i].shape)
    if(S[i].shape[1]<N[i].shape[1]):
        S[i] = np.pad(S[i],((0,0),(0,N[i].shape[1]-S[i].shape[1])),mode='constant')
    elif(S[i].shape[1]>N[i].shape[1]):
        N[i] = np.pad(N[i],((0,0),(0,S[i].shape[1]-N[i].shape[1])),mode='constant')
    # print("After:",S[i].shape,N[i].shape)
    M = np.where((S[i]>N[i]),1,0)
    M_arr.append(M)

In [69]:
for i in range(len(X)):
    X[i] = np.pad(X[i],((0,0),(0,max_len-X[i].shape[1])),mode='constant')
    X[i] = X[i].T
    M_arr[i] = np.pad(M_arr[i],((0,0),(0,max_len-M_arr[i].shape[1])),mode='constant')
    M_arr[i] = M_arr[i].T

In [70]:
print(M_arr[0].shape)
print(M_arr[1].shape)
print(M_arr[2].shape)
print(M_arr[3].shape)

(178, 513)
(178, 513)
(178, 513)
(178, 513)


In [71]:
X = np.dstack(X)
X = np.rollaxis(X,-1)
M_arr = np.dstack(M_arr)
M_arr = np.rollaxis(M_arr,-1)
print(X.shape)
print(M_arr.shape)

(1200, 178, 513)
(1200, 178, 513)


In [72]:
X = tf.convert_to_tensor(X, dtype=tf.float32)
M_arr = tf.convert_to_tensor(M_arr, dtype=tf.float32)
print(X.shape)
print(M_arr.shape)

(1200, 178, 513)
(1200, 178, 513)


In [73]:
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(max_len,513),batch_size=10))
model.add(tf.keras.layers.GRU(513, return_sequences=True, input_shape =(max_len,513), dropout=0.3))
model.add(tf.keras.layers.GRU(513, return_sequences=True, dropout=0.3))
model.add(Dropout(0.3))
model.add(Dense(units=1024,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(units=513,activation='relu'))
model.compile(optimizer='adam',loss = 'mean_squared_error')

In [74]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_31 (GRU)                (10, 178, 513)            1582092   
                                                                 
 gru_32 (GRU)                (10, 178, 513)            1582092   
                                                                 
 dropout_23 (Dropout)        (10, 178, 513)            0         
                                                                 
 dense_25 (Dense)            (10, 178, 1024)           526336    
                                                                 
 dropout_24 (Dropout)        (10, 178, 1024)           0         
                                                                 
 dense_26 (Dense)            (10, 178, 513)            525825    
                                                                 
Total params: 4,216,345
Trainable params: 4,216,345
No

In [ ]:
model.fit(X,M_arr,batch_size=10,epochs=100)

Epoch 1/100
120/120 [==============================] - 264s 2s/step - loss: 0.2025
Epoch 2/100
120/120 [==============================] - 268s 2s/step - loss: 0.1915
Epoch 3/100
120/120 [==============================] - 263s 2s/step - loss: 0.1894
Epoch 4/100
120/120 [==============================] - 268s 2s/step - loss: 0.1866
Epoch 5/100
120/120 [==============================] - 257s 2s/step - loss: 0.1857
Epoch 6/100
120/120 [==============================] - 270s 2s/step - loss: 0.1827
Epoch 7/100
120/120 [==============================] - 257s 2s/step - loss: 0.1811
Epoch 8/100
120/120 [==============================] - 250s 2s/step - loss: 0.1806
Epoch 9/100
120/120 [==============================] - 252s 2s/step - loss: 0.1793
Epoch 10/100
120/120 [==============================] - 251s 2s/step - loss: 0.1773
Epoch 11/100
120/120 [==============================] - 248s 2s/step - loss: 0.1753
Epoch 12/100
120/120 [==============================] - 252s 2s/step - loss: 0.1756
E